In [1]:
import typing
import os

import pandas
import datasets
import trl
import peft

import cltrier_lib

/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
MODEL_SLUG: str = "meta-llama/Llama-3.2-3B-Instruct"
RAW_DATASET: str = "../data/interim/twitter.german.dataset.preds.csv"

SFT_ARGS = trl.SFTConfig(
    num_train_epochs=10,
    per_device_train_batch_size=4,
    packing=True, 
    save_strategy="no",
    output_dir="./sft_results",
    logging_steps=50,
    push_to_hub=True,
    push_to_hub_model_id="Llama-3.2-3B-Instruct-OSN-replies"
)

PEFT_ARGS = peft.LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/transformers/training_args.py:2046: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case simon-muenker/Llama-3.2-3B-Instruct-OSN-replies).
  warnings.warn(


In [4]:
dataset: typing.List[typing.Dict] = [
    cltrier_lib.inference.schemas.Chat(messages=[
        cltrier_lib.inference.schemas.Message(role="system", content=f"You are a social media user with a political {row['leaning']} leaning. Respond to the following Tweet:"),
        cltrier_lib.inference.schemas.Message(role="user", content=row["text_post"]),
        cltrier_lib.inference.schemas.Message(role="assistant", content=row["text_reply"])
    ]).model_dump()
    for _, row in pandas.read_csv(RAW_DATASET, index_col=0).iterrows()
]
dataset[:3]

[{'messages': [{'role': 'system',
    'content': 'You are a social media user with a political neutral leaning. Respond to the following Tweet:'},
   {'role': 'user',
    'content': 'Nicht der #Verbrenner schadet dem #Klima, sondern der fossile Sprit, mit dem er fährt. Wir haben diese Woche den Weg für klimaneutrale #eFuels freigemacht. Damit könnten die mehr als 45 Mio. Diesel- und Benzin-Fahrzeuge auf unseren Straßen in Zukunft klimaneutral unterwegs sein.'},
   {'role': 'assistant', 'content': 'Sie haben wirklich keine Ahnung.'}]},
 {'messages': [{'role': 'system',
    'content': 'You are a social media user with a political neutral leaning. Respond to the following Tweet:'},
   {'role': 'user',
    'content': 'Wo waren die ganzen plötzlichen #Kernkraftbefürworter in #Altparteien, Verbänden &amp; Medien in den letzten Jahren? Warum stimmte die #umfaller: #fdp bis zuletzt im Bundestag gegen Laufzeitverlängerungen? Fakt ist: nur die #AfD lag von Anfang an &amp; jahrelang richtig &amp;

In [5]:
trainer = trl.SFTTrainer(
    MODEL_SLUG,
    args=SFT_ARGS,
    train_dataset=datasets.Dataset.from_pandas(pandas.DataFrame(data=dataset)),
    peft_config=PEFT_ARGS,
)

trainer.train()

trainer.save_model(SFT_ARGS.output_dir)

if SFT_ARGS.push_to_hub:
    trainer.push_to_hub()

/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:185: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.30it/s]
/home/ubuntu/apis/TWON-Agents/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Step,Training Loss
50,2.662000
100,2.246800
150,2.117700
200,2.047600
250,1.993600
300,1.897400
350,1.840700
400,1.821900
450,1.787800
500,1.732200


adapter_model.safetensors:   0%|          | 0.00/18.4M [00:00<?, ?B/s]
training_args.bin: 100%|██████████| 5.56k/5.56k [00:00<00:00, 25.1kB/s]5.0MB/s]
adapter_model.safetensors: 100%|██████████| 18.4M/18.4M [00:01<00:00, 10.8MB/s]
Upload 2 LFS files: 100%|██████████| 2/2 [00:01<00:00,  1.01it/s]
No files have been modified since last commit. Skipping to prevent empty commit.
